In [ ]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

API_URL = os.environ.get('API_BASE_URL', 'http://localhost:8000')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX', '')
PINECONE_HOST = os.environ.get('PINECONE_HOST', '')

print("API_URL:", API_URL)
print("PINECONE_API_KEY:", PINECONE_API_KEY)
print("PINECONE_INDEX:", PINECONE_INDEX)
print("PINECONE_HOST:", PINECONE_HOST)

In [ ]:
# Ingest a file
url = f"{API_URL}/api/v1/ingest"

payload = {
    "files": [
        {
            "type": "PDF",
            "url": "https://arxiv.org/pdf/2210.03629.pdf"
        }
    ],
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "index_name": PINECONE_INDEX,
    "encoder": "cohere",
}

response = requests.post(url, json=payload)

print(response.json())

In [ ]:
# Query the index
query_url = f"{API_URL}/api/v1/query"

query_payload = {
    "input": "What is CoT?",
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "index_name": PINECONE_INDEX,
    "encoder": "cohere",
}

query_response = requests.post(query_url, json=query_payload)

print(query_response.json())


In [ ]:
data = query_response.json().get('data', [])
data

In [ ]:
# Delete the index
query_url = f"{API_URL}/api/v1/delete"

delete_payload = {
    "file_url": "https://arxiv.org/pdf/2210.03629.pdf",
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": PINECONE_API_KEY,
            "host": PINECONE_HOST,
        }
    },
    "index_name": PINECONE_INDEX,
    "encoder": "cohere",
}

delete_response = requests.delete(query_url, json=delete_payload)

print(delete_response.json())